In [1]:
# aa1 stuff
from aa1 import DataLoader
from aa1 import extract_features
from aa1 import check_output

from aa2 import Trainer
from aa2 import parallel_coordinates
from aa2.model import LSTM_fixed_len as model

In [2]:
import torch

device = torch.device('cuda:3')

In [3]:
# assigment 1 stuff
dataset = DataLoader(data_dir="../../DDICorpus/DDICorpus/", device=device)
train_y, val_y, test_y = check_output(dataset.get_y()) 
train_X, val_X, test_X = check_output(extract_features(
                                                        data=dataset.data_df,
                                                        max_sample_length=dataset.max_sample_length,
                                                        id2word=dataset.id2word,
                                                        device=device
                                                        #Add any addtional arguments here
                                                       ))

In [4]:
#set up trainer
model_dump = "../aa2_models/" #you are allowed to change the dump_folder
trainer = Trainer(dump_folder=model_dump) 

In [5]:
# define a set of hyperparamaters
# test at least 5 different sets of hyperparamaters 
set_hyperparamaters = [{"learning_rate": 0.001,
                        "hidden_size": 1000,
                        "number_layers": 3,
                        "optimizer": "adam",
                        "batch_size": 25,
                        "model": 'model1'
                       },
                       {"learning_rate": 0.02,
                        "hidden_size": 100,
                        "number_layers": 5,
                        "optimizer": "SGD",
                        "batch_size": 50,
                        "model": 'model2'
                       },
                       {"learning_rate": 0.1,
                        "hidden_size": 10,
                        "number_layers": 10,
                        "optimizer": "SGD",
                        "batch_size": 25,
                        "model": 'model3'
                       },
                       {"learning_rate": 0.05,
                        "hidden_size": 1000,
                        "number_layers": 16,
                        "optimizer": "SGD",
                        "batch_size": 15,
                        "model": 'model4'
                       },
                       {"learning_rate": 0.02,
                        "hidden_size": 150,
                        "number_layers": 32,
                        "optimizer": "adam",
                        "batch_size": 8,
                        "model": 'model5'
                       }
                        ]

In [6]:
# hyperparamater tuning
# train you model with your set of hyperparamaters 
for hp in set_hyperparamaters:
    trainer.train(train_X, train_y, val_X, val_y, model, hp)

Total loss in epoch 0 is 399.4423522949219.
accuracy: 0.7144987564031194 precision: 0.9362996876257482 recall: 0.7144987564031194 f1_score: 0.7648544428436505
Total loss in epoch 0 is 1963.6229248046875.


/home/guspfame@GU.GU.SE/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.17645286333712967 precision: 0.031135612979871764 recall: 0.17645286333712967 f1_score: 0.05293133953799456
Total loss in epoch 0 is 757.1072998046875.
accuracy: 0.7823271349264311 precision: 0.9617625402306739 recall: 0.7823271349264311 f1_score: 0.8478219463125537
Total loss in epoch 0 is 1368.154296875.
accuracy: 0.7091209021312237 precision: 0.9616498267085415 recall: 0.7091209021312237 f1_score: 0.7963231922149068
Total loss in epoch 0 is 1137.895263671875.


/home/guspfame@GU.GU.SE/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.7478945674989658 precision: 0.9381500076400355 recall: 0.7478945674989658 f1_score: 0.8202371094408032


In [7]:
# create a parallel coordination plot over hyperparamaters
# add path to models and change metric to what ever metric you have chosen to use/want to use
parallel_coordinates(model_dump, metric="f1:score")

models found: ['../aa2_models/model1.pt', '../aa2_models/model2.pt', '../aa2_models/model3.pt', '../aa2_models/model4.pt', '../aa2_models/model5.pt']


TypeError: 'set' object does not support item assignment

In [ ]:
# Based on the Parallel Coordination plot define 3 new hyperparamaters set thay you 
# think are worth testing
set_hyperparamaters_2 = [
                        # Example:
                        # {
                        #   "learning_rate": 0.1,
                        #   "number_layers": 3,
                        #   "optimizer": "adam"
                        # }
                        ]

In [ ]:
# train models for the new hyperparamaters
for hp in set_hyperparamaters_2:
    trainer.train(train_X, train_y, val_X, val_y, YOUR_MODEL_CLASS, hp)

In [ ]:
# create a parallel coordination plot over hyperparamaters again
parallel_coordinates(model_dump, metric="loss")

In [ ]:
#choose the best model base on the last parallel coordination plot
best_model_path = "PATH TO THE BEST MODEL"
scores = trainer.test(test_X, test_y, YOUR_MODEL_CLASS, best_model_path)